In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
from torchvision.io import read_image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
train_data = 'train.csv'
test_data = 'test.csv'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [33]:
class HandDataset(Dataset):
    def __init__(self, csv_file, transform=None, target_transform=None):
        self.kind = csv_file.split('.')[0]
        self.img_labels = pd.read_csv(csv_file)
        if self.kind == 'train':
            self.img_labels['label'].loc[self.img_labels['label'] == '10-1'] = '0'
            self.img_labels['label'].loc[self.img_labels['label'] == '10-2'] = '10'
            self.img_labels['label'] = self.img_labels['label'].astype(int)
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.kind,self.img_labels.iloc[idx,0])
        img = read_image(img_path)
        if self.kind == 'test':
            return img / 255
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            img = self.transform(img)
        else:
            img = img / 255
        return img, label

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.conv3 = nn.Conv2d(16, 32, 3)
        # self.conv4 = nn.Conv2d(32, 64, 3)
        # self.conv5 = nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(32 * 26 * 26, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.head = nn.Linear(32, 11)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # x = self.pool(F.relu(self.conv4(x)))
        # x = self.pool(F.relu(self.conv5(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.head(x)
        return x

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, data in enumerate(dataloader):
        X, y = data
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for data in dataloader:
            X, y = data
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 30

model = Net()
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [75]:
train_dataset = HandDataset(train_data)
test_dataset = HandDataset(test_data)

In [76]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

In [27]:
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(train_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------
loss: 2.407753 [    0/  858]
loss: 2.404529 [  640/  858]
Test Error: 
 Accuracy: 9.6%, Avg loss: 2.394379 

Epoch 2
-------------------
loss: 2.385957 [    0/  858]
loss: 2.393835 [  640/  858]
Test Error: 
 Accuracy: 9.6%, Avg loss: 2.394140 

Epoch 3
-------------------
loss: 2.389555 [    0/  858]
loss: 2.402508 [  640/  858]
Test Error: 
 Accuracy: 9.7%, Avg loss: 2.392388 

Epoch 4
-------------------
loss: 2.386420 [    0/  858]
loss: 2.399643 [  640/  858]
Test Error: 
 Accuracy: 9.7%, Avg loss: 2.395076 

Epoch 5
-------------------
loss: 2.379123 [    0/  858]
loss: 2.388311 [  640/  858]
Test Error: 
 Accuracy: 9.6%, Avg loss: 2.394218 

Epoch 6
-------------------
loss: 2.388061 [    0/  858]
loss: 2.399062 [  640/  858]
Test Error: 
 Accuracy: 9.6%, Avg loss: 2.391488 

Epoch 7
-------------------
loss: 2.383374 [    0/  858]
loss: 2.382449 [  640/  858]
Test Error: 
 Accuracy: 13.5%, Avg loss: 2.369652 

Epoch 8
-------------------
loss: 2.3597

In [77]:
correct = 0
total = 0
summit = []
with torch.no_grad():
    for data in test_dataloader:
        images = data.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        v = str(int(predicted.cpu()))
        if v == '0':
            v = '10-1'
        elif v== '10':
            v = '10-2'
        summit.append(v)

In [78]:
submission = pd.read_csv('sample_submission.csv')

In [79]:
submission['label'] = summit

In [80]:
submission

,file_name,label
0,001.png,6
1,002.png,8
2,003.png,1
3,004.png,6
4,005.png,9
...,...,...
210,211.png,5
211,212.png,8
212,213.png,3
213,214.png,6


In [81]:
submission.to_csv('commit_1.csv', index=0)

In [ ]:
# x,y = next(iter(train_dataloader))

In [ ]:
# l = k.cpu().numpy().astype(int)

In [ ]:
# plt.imshow(np.transpose(l, (1, 2, 0)))